## install required packages
run the following command line in terminal

       pip install -r requirements.txt

## Import packages

In [14]:
import pandas as pd
import numpy as np
import os
import json
import obonet
import inflect
import networkx as nx
import matplotlib.pyplot as plt
from openai import OpenAI
from pydantic import BaseModel
import google.generativeai as genai
from dotenv import load_dotenv
load_dotenv()


Matplotlib is building the font cache; this may take a moment.


True

## Define output format

In [3]:
class cellTypeFormat(BaseModel):
       cellType: str

## Read cell clusters file 
all.csv

In [4]:
def read_cluster(path):
      dataframe = pd.read_csv(path)
      print(dataframe['marker'])
      return dataframe 
gene_list = read_cluster('./Data/all.csv')
print(gene_list.iloc[0])

0       MS4A1, TNFRSF13B, IGHM, IGHD, AIM2, CD79A, LIN...
1       MS4A1, COCH, AIM2, BANK1, SSPN, CD79A, TEX9, R...
2       IGHM, IGHD, CD79A, IL4R, MS4A1, CXCR4, BTG1, T...
3       IGHA2, MZB1, TNFRSF17, DERL3, TXNDC5, TNFRSF13...
4       GZMH, CD4, FGFBP2, ITGB1, GZMA, CST7, GNLY, B2...
                              ...                        
1125    KLRD1,NKG7,XCL2,CTSW,XCL1,GNLY,GZMA,KLRB1,KLRC...
1126    JCHAIN,MZB1,DERL3,IGHG2,IGHA2,TNFRSF17,SDC1,FC...
1127    TFF3,PKHD1L1,PROX1,NTS,FLT4,RELN,GPR182,STAB2,...
1128    VWF,ACKR1,PLVAP,PECAM1,AQP1,RAMP3,CLEC14A,SOX1...
1129    MSLN,UPK3B,MUC16,CALB2,KLK11,BICDL1,CHAC1,TGM1...
Name: marker, Length: 1130, dtype: object
dataset                                                               Azimuth
tissue                                                                   PBMC
marker                      MS4A1, TNFRSF13B, IGHM, IGHD, AIM2, CD79A, LIN...
manual_annotation                                         Intermediate B cell
manual_C

## Split genelist

In [5]:
# Generate the user message for each chunk
def generate_user_message(gene_list:  pd.DataFrame, max_chunk_size=100) -> list:
    # Split the gene_list into chunks
    chunks = [group for _, group in gene_list.groupby(gene_list.index // max_chunk_size)]
    messages = []
    for chunk in chunks:
        # Construct the user message for each chunk
        user_message = (
            chunk['tissue'] + ' : '
            + chunk['marker']
        )
        messages.append(user_message)
    return messages

# Generate messages with a tunable chunk size
messages = generate_user_message(gene_list, max_chunk_size=10)  # Adjust max_chunk_size as needed
print(messages[0])

0    PBMC : MS4A1, TNFRSF13B, IGHM, IGHD, AIM2, CD7...
1    PBMC : MS4A1, COCH, AIM2, BANK1, SSPN, CD79A, ...
2    PBMC : IGHM, IGHD, CD79A, IL4R, MS4A1, CXCR4, ...
3    PBMC : IGHA2, MZB1, TNFRSF17, DERL3, TXNDC5, T...
4    PBMC : GZMH, CD4, FGFBP2, ITGB1, GZMA, CST7, G...
5    PBMC : TCF7, CD4, CCR7, IL7R, FHIT, LEF1, MAL,...
6    PBMC : MKI67, TOP2A, PCLAF, CENPF, TYMS, NUSAP...
7    PBMC : IL7R, TMSB10, CD4, ITGB1, LTB, TRAC, AQ...
8    PBMC : IL7R, CCL5, FYB1, GZMK, IL32, GZMA, KLR...
9    PBMC : RTKN2, FOXP3, AC133644.2, CD4, IL2RA, T...
dtype: object


## Gemeni ai

### model configuration

In [7]:
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
gemeni_model = genai.GenerativeModel("gemini-1.5-flash")
gemeni_config = genai.GenerationConfig(response_mime_type="application/json",response_schema=list[cellTypeFormat])

### Send prompt to gemini

In [24]:
earlyStop = 2
Prompt = "Identify cell types using the following tissue name and markers separately for each row. Only provide the cell type name. Do not show numbers before the name. Some can be a mixture of multiple cell types."
def annotateCell_Gemini(messages: list)->list:
    global gemeni_model, gemeni_config 
    responses = []
    i = 0
    for message in messages:
           i+=1
           if i> 2:
                  break
           prompt = Prompt ## create a copy of the prompt
           for cluster in message:
                  prompt += cluster
           response = gemeni_model.generate_content(
                     prompt,
                     generation_config=gemeni_config
              )
           responses.append(response.text)
    return responses


In [25]:
annotateCell_Gemini_results = annotateCell_Gemini(messages)
print(annotateCell_Gemini_results)

['[{"cellType": "B cells"}, {"cellType": "B cells"}, {"cellType": "B cells"}, {"cellType": "B cells"}, {"cellType": "Cytotoxic T cells"}, {"cellType": "T cells"}, {"cellType": "Proliferating cells"}, {"cellType": "T cells"}, {"cellType": "T cells"}, {"cellType": "T regulatory cells"}]', '[{"cellType": "T cells"}, {"cellType": "T cells"}, {"cellType": "T cells"}, {"cellType": "T cells"}, {"cellType": "Monocytes"}, {"cellType": "NK cells"}, {"cellType": "NK cells"}, {"cellType": "Dendritic cells"}, {"cellType": "B cells"}, {"cellType": "Macrophages"}, {"cellType": "B cells"}]']


## GPT

### model configuration

In [2]:
client = OpenAI()

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/qwer/files/developer/Medicine_and_EECS/MedandEECS_env/lib/python3.12/site-packages/IPython/core/interactiveshell.py", line 3577, in run_code
  File "/var/folders/jd/741sx0vn7h98nzy_4kpsbgg40000gn/T/ipykernel_93815/1371122800.py", line 1, in <module>
    client = OpenAI()
             ^^^^^^
NameError: name 'OpenAI' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/qwer/files/developer/Medicine_and_EECS/MedandEECS_env/lib/python3.12/site-packages/pygments/styles/__init__.py", line 45, in get_style_by_name
ModuleNotFoundError: No module named 'pygments.styles.default'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/qwer/files/developer/Medicine_and_EECS/MedandEECS_env/lib/python3.12/site-packages/IPython/core/interactiveshell.py", line 2168, in showtraceback
  File "/Users/qwer/files/

### upload file

In [ ]:
def upload_cell_cluster(filename):
       global client
       client.files.create(
              file=open(filename, "rb"),
              purpose="assistant"
       )
       print(client.files.list())

### send prompt to GPT

In [ ]:
def annotateCell_GPT(messages: list)->list:
    global client
    responses = []
    for message in messages:
        completion = client.beta.chat.completions.parse(
            model="gpt-4o-2024-08-06",
            messages=[
                {"role": "system", "content":  "Identify cell types using the following tissue name and markers separately for each row. "
            + "Only provide the cell type name. "
            + "Do not show numbers before the name. "
            + "Some can be a mixture of multiple cell types."},
                {"role": "user", "content": message}
            ],
            response_format=cellTypeFormat,
        )
        responses.append(completion.choices[0].message.parsed)
    return responses


## Scoring system

### result parsing

In [11]:
def json_parsing(annotateCell_results: json)->list:
       # Parse the strings into Python objects
       parsed_results = [json.loads(item.strip()) for item in annotateCell_results]
       # Flatten the results to get a list of all cell types
       cell_types = [entry['cellType'] for result in parsed_results for entry in result]
       return cell_types

In [12]:
cell_types = json_parsing(annotateCell_Gemini_results)

### CL correspondence
match the cell_types to get CLID from Cell Ontology

In [20]:
# URL for Cell Ontology (CO) OBO file
CO_URL = 'http://purl.obolibrary.org/obo/CL.obo'
def load_ontology(url):
    graph = obonet.read_obo(url)
    return graph

inflector = inflect.engine()

def lookup_clid_by_name(graph: nx.graph, cell_name: str)->str:
   
    normalized_name = inflector.singular_noun(cell_name.lower()) or cell_name.lower()

    # Search for the term by normalized name
    for node, data in graph.nodes(data=True):
        if 'name' in data:
            # Singularize and normalize the graph's 'name'
            graph_name = inflector.singular_noun(data['name'].lower()) or data['name'].lower()

            # Compare normalized names
            if graph_name == normalized_name:
                return node  # Return the term ID if a match is found

    return None

# Load the Cell Ontology
graph = load_ontology(CO_URL)


In [21]:
clids = []
for cell_type in cell_types:
  clids.append(lookup_clid_by_name(graph, cell_type))

print(clids)

['CL:0000236', 'CL:0000236', 'CL:0000236', 'CL:0000236', 'CL:0000910', 'CL:0000084', None, 'CL:0000084', 'CL:0000084', 'CL:0000084', None, 'CL:0000084', None, 'CL:0000910', None, None, None, None, None, None, 'CL:0000236']


### Calculate difference with two CLID
change the graph from Cell Ontology to undirected graph
and calculate the distance between two nodes ( manual CLID and LLM annotated CLID)

In [ ]:
ugraph = graph.to_undirected()

def calculate_difference(graph, clid_1, clid_2):

    try:
        return nx.shortest_path_length(graph, source=node1, target=node2)
    except nx.NetworkXNoPath:
        return None  # Return None if no path exists


In [23]:
diff_result=[]
for i in range(len(clids)):
  clid_1 =clids[i] #LLM annotated CLID
  clid_2 =gene_list["manual_CLID"][i] # manual annotated CLID
  difference = calculate_difference(ugraph, clid_1, clid_2)
  diff_result.append(difference)

print(diff_result)

NodeNotFound: Source nan is not in G